In [1]:
import copy
from pathlib import Path
import warnings
import random

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

import matplotlib.pyplot as plt

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
input_length = 168
output_length = 24

show_graph = False

In [3]:
# model_paths = ["Save_File/experiment/A7/4108.ckpt",
#                "Save_File/experiment/A7/A7.ckpt"]

# building_num = [41,41]

model_paths = ["Save_File/experiment/D2/D2.ckpt",
               "Save_File/experiment/D2/1501.ckpt",
               "Save_File/experiment/D2/1502.ckpt",
               "Save_File/experiment/D2/1503.ckpt",
               "Save_File/experiment/D2/1504.ckpt",
               "Save_File/experiment/D2/1505.ckpt",
               "Save_File/experiment/D2/1506.ckpt",
               "Save_File/experiment/D2/1507.ckpt",
               "Save_File/experiment/D2/1508.ckpt",
               "Save_File/experiment/D2/1509.ckpt",
               "Save_File/experiment/D2/1510.ckpt",
               "Save_File/experiment/D2/1511.ckpt"]

building_num = [15 for i in range(12)]
# building_num = [46 for i in range(12)]
# building_num = [47 for i in range(12)]

data_path = "./Refined_Data/Grouped_Data/Input_Data2.csv"

In [4]:
dataframes = pd.read_csv(data_path, parse_dates = ["date_time"])

dataframes['num']     =   dataframes['num'].apply(str)
dataframes['Week']    =   dataframes['Week'].apply(str)
dataframes['24Hour']  =   dataframes['24Hour'].apply(str)
dataframes['holiday'] =   dataframes['holiday'].apply(str)
dataframes['Weekend'] =   dataframes['Weekend'].apply(str)
dataframes['energy_group'] = dataframes['energy_group'].apply(str)

# random_choice = random.randrange(2,13)

# start_point = -168 * random_choice - 1
# duration = 168

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

all_smape = [[] for i in range(len(model_paths))]
step_smape = [[] for i in range(2, 13)]
step_mean_smape = []

In [5]:
for random_choice in range(2, 13):

    print(f"selected time slice : {random_choice}")
    start_point = -168 * random_choice - 1
    duration = 168

    for i in range(len(model_paths)):
        now_building_dataframe = dataframes.loc[dataframes["num"] == str(building_num[i])].copy()
        now_building_dataframe = now_building_dataframe[start_point:start_point+input_length+duration]
        
        now_best_tft = TemporalFusionTransformer.load_from_checkpoint(model_paths[i])

        original = now_building_dataframe.iloc[0:input_length]["kWH"].tolist()
        prediction = now_building_dataframe.iloc[0:input_length]["kWH"].tolist()  

        total_cycle = ((len(now_building_dataframe) - input_length - output_length) // output_length) + 1

        for k in range(total_cycle):
            now_start_point = k*output_length

            encoder_data = now_building_dataframe.iloc[now_start_point:now_start_point+input_length].copy()
            decoder_data = now_building_dataframe.iloc[now_start_point+input_length:now_start_point+input_length+output_length].copy()
            new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)

            raw_predictions = now_best_tft.predict(new_prediction_data, mode="prediction").numpy().tolist()[0]
            originals = now_building_dataframe.iloc[now_start_point+input_length : now_start_point+input_length+output_length]["kWH"].tolist()

            prediction.extend(raw_predictions)
            original.extend(originals)

            # for next step, change dataframe's original value to predicted value
            for p in range(output_length):
                now_building_dataframe.iloc[now_start_point+input_length+p, now_building_dataframe.columns.get_loc("kWH")] = raw_predictions[p]
            
        if show_graph:
            plt.rcParams["figure.figsize"] = (17,5)
            fig = plt.figure()
            graph = fig.add_subplot(1, 1, 1)
            graph.plot(prediction, color='blue')
            graph.plot(original, color='red')
            plt.show()

        smape_loss = smape(np.array(prediction[168:]), np.array(original[168:]))
        print(f"building : {model_paths[i]}, score : {smape_loss}")
        all_smape[i].append(smape_loss)
        step_smape[random_choice-2].append(smape_loss)
    print(f"this slice : {random_choice}, score : {np.mean(np.array(step_smape[random_choice-2]))}")
    step_mean_smape.append(np.mean(np.array(step_smape[random_choice-2])))

selected time slice : 2
building : Save_File/experiment/D2/D2.ckpt, score : 7.502667147214297
building : Save_File/experiment/D2/1501.ckpt, score : 3.6399701788634897
building : Save_File/experiment/D2/1502.ckpt, score : 5.575539136909946
building : Save_File/experiment/D2/1503.ckpt, score : 6.080167112420271
building : Save_File/experiment/D2/1504.ckpt, score : 6.616869390854831
building : Save_File/experiment/D2/1505.ckpt, score : 6.762505568533123
building : Save_File/experiment/D2/1506.ckpt, score : 5.221791910376999
building : Save_File/experiment/D2/1507.ckpt, score : 5.634637078077233
building : Save_File/experiment/D2/1508.ckpt, score : 6.3590164622902074
building : Save_File/experiment/D2/1509.ckpt, score : 4.207047176708442
building : Save_File/experiment/D2/1510.ckpt, score : 7.41600245806372
building : Save_File/experiment/D2/1511.ckpt, score : 8.176643442788391
this slice : 2, score : 6.099404755258412
selected time slice : 3
building : Save_File/experiment/D2/D2.ckpt, sco

In [ ]:
print(f"total score mean : {np.mean(np.array(step_mean_smape))}")
for i in range(len(all_smape)):
    print(f"index : {i}, totoal score : {np.mean(np.array(all_smape[i]))}, min : {np.min(np.array(all_smape[i]))}, max : {np.max(np.array(all_smape[i]))}")

total score mean : 7.470932439035545
index : 0, totoal score : 8.710177949488651, min : 7.075765632911859, max : 9.557796226639436
index : 1, totoal score : 3.3688929582585256, min : 2.630444473251455, max : 3.944647064152246
index : 2, totoal score : 6.922288529244438, min : 5.519147357253473, max : 8.612788038164673
index : 3, totoal score : 8.144843136140329, min : 6.080167112420271, max : 10.003323825738896
index : 4, totoal score : 8.392229980683048, min : 6.173086020064831, max : 9.47050790986143
index : 5, totoal score : 8.348289562118698, min : 6.762505568533123, max : 10.085264356684489
index : 6, totoal score : 6.57899971302495, min : 5.221791910376999, max : 7.52513109828435
index : 7, totoal score : 7.842079960142322, min : 5.521049997119174, max : 11.398141199710887
index : 8, totoal score : 7.61128532309265, min : 5.309624099437114, max : 8.914002371973718
index : 9, totoal score : 4.95761041747428, min : 4.121874266820836, max : 5.663364929435399
index : 10, totoal score